In [1]:
# theano imports
import theano
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
from theano.tensor.nnet.conv import conv2d
# from theano.tensor.signal.downsample import max_pool_2d
from theano.tensor.signal.pool import pool_2d as max_pool_2d
from theano.tensor.nnet import batch_normalization

# other imports
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Getting our data in 

In [3]:
# helper function for loading in data of a specific encoding window
def get_data_tensor(n = 5):
    filename = 'conv_data/' + str(n) + '_tensor.p'
    
    with open(filename, 'rb') as f:
        loaded_data = pickle.load(f)
    
    return loaded_data

In [4]:
# read our data in 

n_window = 13
n_aminos = 21

loaded_data = get_data_tensor(n = n_window)
    
labels = pd.read_csv('one_hot_labels.csv')

In [5]:
loaded_data[:2]    

array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0

In [6]:
one_hot = labels.values

In [7]:
one_hot[:2]

array([[0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [8]:
# # trying to scale data
# sc = StandardScaler()
# sc.fit(loaded_data[0])

# # now scale each pt
# scaled_data = np.asarray([sc.transform(d) for d in loaded_data])

In [9]:
xTrain, xTest, yTrain, yTest = train_test_split(loaded_data, one_hot)

In [10]:
print xTrain.shape, xTest.shape, yTrain.shape, yTest.shape

(101110, 13, 21) (33704, 13, 21) (101110, 6) (33704, 6)


In [11]:
xTrain = xTrain.reshape(-1, 1, n_window, n_aminos)
xTest = xTest.reshape(-1, 1, n_window, n_aminos)

In [12]:
print xTrain.shape, xTest.shape, yTrain.shape, yTest.shape

(101110, 1, 13, 21) (33704, 1, 13, 21) (101110, 6) (33704, 6)


## Now time to declare some Theano functions

In [13]:
srng = RandomStreams()

def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def glorot_init_weights(shape):
    (h, w) = shape
    # 0.25 for sigmoid, 0.1 for softmax, 1.0 for tanh/relu
    normalizer = 2.0 * (6**0.5) / ((h + w)**0.5) * 1.0  #factors: 0.1 correct for uni[0,1], glo, glo, softmax deriv
    return theano.shared(floatX((np.random.random_sample(shape) - 0.5) * normalizer))

def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

def activate(X):
    return T.nnet.relu(X)

def rectify(X):
#     return T.maximum(X, 0.)
    return T.maximum(X, 0.01*X)  #leaky rectifier

def ELU(X, alpha=0.1):
    return T.switch(X > 0, X, alpha * (T.exp(X) - 1))
    
def softmax(X):
    e_x = T.exp(X - X.max(axis=1).dimshuffle(0, 1, 'x', 'x'))
    return e_x / e_x.sum(axis=1).dimshuffle(0, 1, 'x', 'x')

def dropout(X, p=0.0):
    if p > 0:
        retain_prob = 1 - p
        X *= srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
        X /= retain_prob
    return X

def RMSprop(cost, params, lr=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        updates.append((p, p - lr * g))
    
    return updates

def shuffle(x, y):
    # helper function to shuffle indicies each loop 
    index = np.random.choice(len(x), len(x), replace=False)
    return x[index], y[index]

## Trying to implement batch normalization

In [14]:
# hoping to get batch normalization implemented below
# data, input weights, gamma input, beta input, hidden weights, hidden bias, gamma hidden, beta hidden, 
# output weights, output bias, gamma output, beta output, p_drop, p_hidden_drop
def model_bn(X, wi, gi, bbi, wh, bh, gh, bbh, wo, bo, go, bbo, p_drop_conv, p_drop_hidden):

    # --------------------------------------------
    
    layer_1 = conv2d(X, wi, border_mode='valid')
    layer_1 = layer_1.reshape((-1, 256))
    layer_1 = batch_normalization(layer_1, gamma=gi, beta=bbi, 
                                 mean=layer_1.mean((1), keepdims=True), 
                                  std = T.ones_like(layer_1.std((1), keepdims=True)), 
                                  mode='high_mem')

#     layer_1 = dropout(layer_1, p_drop_conv)

    # --------------------------------------------
    
    layer_2 = T.dot(layer_1, wh) + bh
#     layer_2 = batch_normalization(layer_2, gamma=gh, beta=bbh, 
#                                  mean=X.mean((1, ), keepdims=True), 
#                                   std = T.ones_like(layer_2.var((1,), keepdims=True)), 
#                                   mode='high_mem')
    
    layer_2 = activate(layer_2)
    layer_2 = dropout(layer_2, p_drop_hidden)
    
    # --------------------------------------------
    
    layer_3 = T.dot(layer_2, wo) + bo
#     layer_3 = batch_normalization(layer_3, gamma=go, beta=bbo, 
#                                  mean=X.mean((1, ), keepdims=True), 
#                                   std = T.ones_like(layer_3.var((1,), keepdims=True)), 
#                                   mode='high_mem')
    
    layer_3 = dropout(layer_3, p_drop_hidden)
    
    # --------------------------------------------
    
    # thinks it's getting a 4D Tensor ???
#     pyx = softmax(layer_3)
    pyx = T.nnet.softmax(layer_3)
    return layer_1, layer_2, layer_3, pyx

In [16]:
X = T.ftensor4()
Y = T.fmatrix()

# define mini-batch size
mbs = 64

# define number of desired features out of convolution
n_conv = 256

# define hidden layer depth
h_depth = 600

# define output layer size
o_depth = 6

# --------------------------- FOR BATCH NORMALIZATION (NOT WORKING) -----------------------

# initialize weight matrices: wi, gi, bbi, wh, bh, gh, bbh, wo, bo, go, bbo

# input parameters
wi = init_weights((n_conv, 1, n_window, n_aminos))
gi = theano.shared(floatX(np.ones((mbs, n_conv))))
bbi = theano.shared(floatX(np.zeros((mbs, n_conv))))

# hidden parameters
wh = glorot_init_weights((n_conv, h_depth))
bh = theano.shared(floatX(np.zeros(h_depth))) # can remove later
gh = theano.shared(floatX(np.ones((mbs, h_depth))))
bbh = theano.shared(floatX(np.zeros((mbs, h_depth))))

# output parameters
wo = glorot_init_weights((h_depth, o_depth))
bo = theano.shared(floatX(np.zeros(o_depth)))
go = theano.shared(floatX(np.ones((mbs, o_depth))))
bbo = theano.shared(floatX(np.zeros((mbs, o_depth))))

#modeling and parameters for Gradient Descent
noise_l1, noise_l2, noise_l3, noise_py_x = model_bn(X, wi, gi, bbi, wh, bh, gh, bbh, wo, bo, go, bbo, 0.2, 0.5)
l1, l2, l3, py_x = model_bn(X, wi, gi, bbi, wh, bh, gh, bbh, wo, bo, go, bbo, 0., 0.)
# params = [wi, gi, bbi, wh, bh, gh, bbh, wo, bo, go, bbo]

params = [wi, gi, bbi, wh, bh, wo, bo]

# --------------------------- FOR BATCH NORMALIZATION (NOT WORKING) -----------------------

y_x = T.argmax(py_x, axis=1)
cost = T.mean(T.nnet.categorical_crossentropy(noise_py_x, Y))
updates = RMSprop(cost, 
                  params, 
                  lr=1e-4) #lr=1e-7 <--- way too small of a LR

train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True)
predict = theano.function(inputs=[X], outputs=y_x, allow_input_downcast=True)

In [63]:
import time

# number of training iterations to perform
n_train = 31

# store our results
costs = []
test_scores = []
train_scores = []

# borrowing some mini-batching for

Trainbatches = zip(range(0, len(xTrain), mbs), range(mbs, len(xTrain), mbs))
Testbatches  = zip(range(0, len(xTest), mbs), range(mbs, len(xTest), mbs))
y_codes      = np.argmax(yTest, axis=1)

t_start = time.time()
t_end = t_start + 60 * 100
count = 1


while time.time() < t_end:

    for start, end in Trainbatches:
#             index = np.random.choice(xTrain.shape[0], batch_size, replace=False)
        cost = train(xTrain[start:end], yTrain[start:end])
        if time.time() > t_end:
            break


    xTrain, yTrain = shuffle(xTrain, yTrain)
    xTest, yTest   = shuffle(xTest, yTest)


    tr, trr = [], []
    
    for start, end in Testbatches:    
        tr  = [np.mean(np.argmax(yTest[start:end], axis=1) == predict(xTest[start:end]))]
        trr = [np.mean(np.argmax(yTrain[start:end], axis=1) == predict(xTrain[start:end]))]
    print "Round: %-5s Test: %-14s Train: %-8s" % (count, np.mean(tr), np.mean(trr))

#     if count == 15:
#         cost = T.mean(T.nnet.categorical_crossentropy(noise_py_x, Y))
#         params = [w, w2, w3, w4, w_o]
#         updates = RMSprop(cost, params, lr=0.0001)

#         train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, 
#                                 allow_input_downcast=True)

#         predict = theano.function(inputs=[X], outputs=y_x, allow_input_downcast=True)

    count += 1

print "Time:",  round(((time.time() - t_start)/60.0), 2), 'minutes'


Round: 1     Test: 0.71875        Train: 0.703125
Round: 2     Test: 0.625          Train: 0.59375 
Round: 3     Test: 0.609375       Train: 0.59375 
Round: 4     Test: 0.53125        Train: 0.546875
Round: 5     Test: 0.65625        Train: 0.703125
Round: 6     Test: 0.484375       Train: 0.71875 
Round: 7     Test: 0.671875       Train: 0.546875
Round: 8     Test: 0.5            Train: 0.65625 
Round: 9     Test: 0.5625         Train: 0.71875 
Round: 10    Test: 0.6875         Train: 0.71875 
Round: 11    Test: 0.640625       Train: 0.609375
Round: 12    Test: 0.625          Train: 0.78125 
Round: 13    Test: 0.65625        Train: 0.59375 
Round: 14    Test: 0.71875        Train: 0.6875  
Round: 15    Test: 0.59375        Train: 0.703125
Round: 16    Test: 0.53125        Train: 0.703125
Round: 17    Test: 0.65625        Train: 0.59375 
Round: 18    Test: 0.6875         Train: 0.5625  
Round: 19    Test: 0.71875        Train: 0.640625
Round: 20    Test: 0.71875        Train: 0.640625


KeyboardInterrupt: 

In [65]:
plt.plot(xrange(n_train), train_scores, label='train')
plt.plot(xrange(n_train), test_scores, label='test')
plt.xlabel('Steps')
plt.ylabel('Accuracy')
plt.legend(loc=2);

In [ ]:
# plot our results
plt.plot(xrange(n_train), costs)
plt.xlabel('Steps')
plt.ylabel('Cost');

In [ ]:
# AMINO MAPS (n_window, 21)
aminos = ['-','A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
# positions = ['-1', '-2', '0', '1', '2']

positions = sorted(map(str, range(-(n_window-1)//2, (n_window+1)//2, 1)))

for conv in wi.eval()[:5]:
    c = conv.reshape(n_window, n_aminos)
    plt.imshow(c, cmap='Greys')
    plt.xticks(range(len(aminos)), aminos)
    plt.yticks(range(len(positions)), positions)
    plt.colorbar()
    plt.show()

http://deeplearning.net/software/theano/library/tensor/nnet/nnet.html#tensor.nnet.softmax
http://deeplearning.net/software/theano/library/tensor/nnet/conv.html#theano.tensor.nnet.conv.conv2d